<a href="https://colab.research.google.com/github/shoobi12/face-detection-/blob/main/My_attendence_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install face-recognition
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=468ccb10e3483d07ddec7c3f7ec2080463bc62cf8a6431f148a38f3c2fbd9b41
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import face_recognition
import numpy as np
import os
import cv2
import pandas as pd
from datetime import datetime
import csv


path = "/content/drive/MyDrive/images"  # Path to known faces

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Wait for 3 seconds
            await new Promise(resolve => setTimeout(resolve, 4000));

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# Use the function and capture photo automatically
try:
    filename = take_photo()
    print('Saved to {}'.format(filename))
except Exception as err:
    print(str(err))

# Process the captured image for face recognition
known_face_encodings = []
known_face_names = []



# Load and encode known faces
# Load and encode known faces
for filename in os.listdir(path):
    if filename.endswith(".jpeg") or filename.endswith(".jpg"):
        current_image = face_recognition.load_image_file(f"{path}/{filename}")
        current_image_encodings = face_recognition.face_encodings(current_image)
        # Check if any face is detected
        if current_image_encodings:
            current_image_encoded = current_image_encodings[0]
            known_face_encodings.append(current_image_encoded)
            known_face_names.append(os.path.splitext(filename)[0])
        else:
            print(f"No faces found in {filename}.")


# Load and encode the captured image
captured_image = face_recognition.load_image_file('photo.jpg')  # Use the captured photo
captured_image_encodings = face_recognition.face_encodings(captured_image)

if captured_image_encodings:
    captured_face_encoding = captured_image_encodings[0]
    matches = face_recognition.compare_faces(known_face_encodings, captured_face_encoding)
    name = "Unknown"

    face_distances = face_recognition.face_distance(known_face_encodings, captured_face_encoding)
    best_match_index = np.argmin(face_distances)
    if matches[best_match_index]:
        name = known_face_names[best_match_index]

    print(f"Detected: {name}")
else:
    print("No faces found in the image.")



# After recognizing the face
attendance_file = 'attendance.csv'

# Check if the file exists, and create it with headers if it doesn't
if not os.path.exists(attendance_file):
    with open(attendance_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Name', 'Time', 'Status'])

# Write the attendance record
with open(attendance_file, mode='a', newline='') as file:
    writer = csv.writer(file)
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    writer.writerow([name, current_time, "Present"])

print(f"Attendance marked for {name} at {current_time}")

<IPython.core.display.Javascript object>

NotAllowedError: Permission denied
No faces found in the image.


NameError: name 'name' is not defined